## Test the jaxpower codes

In [5]:
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table,join,Column

import jax
from jaxpower import (ParticleField, FKPField, compute_fkp2_normalization, compute_fkp2_shotnoise, BinMesh2SpectrumPoles, get_mesh_attrs, compute_mesh2_spectrum)

sys.path.append('/global/homes/s/shengyu/desi_y3_redshift_errors/main/')
from helper import REDSHIFT_BIN_OVERALL, REDSHIFT_BIN_LSS, REDSHIFT_ABACUSHF_v1, NRAN, TRACER_CUTSKY_INFO
from helper import GET_REPEATS_DV, GET_CTHR, GET_REPEATS_NUMBER
from cat_tools import read_positions_weights

BASE_DIR = '/pscratch/sd/s/shengyu/galaxies/catalogs/Y3/AbacusHF-v1'
REPEAT_DIR = '/pscratch/sd/s/shengyu/repeats/DA2/loa-v1'

def zfmt(x):
    return f"{x:.3f}".replace(".", "p")

BOXSIZE = 2000
NSPLITS = 10
ells = (0, 2, 4)
kedges   = np.arange(0.,0.4001,0.001) # for PK
# kmin, kmax, kbin, lenk = 0.02, 0.2, 0.005, 36

In [6]:
tracer = 'QSO'
indz = 0
data_args = {'tracer':tracer, 'indz': indz, 'mock_id03': '000', 'mock_dir':'000', 'domain':'cutsky'}

data_positions, _ = read_positions_weights(data_args, use_zerr=None)
# positions1, _ = read_positions_weights(data_args, use_zerr='global')
# positions2, _ = read_positions_weights(data_args, use_zerr='bin')
random_positions_all, _ = read_positions_weights(data_args, use_random=True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/global/homes/s/shengyu/.conda/envs/hod_env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/global/homes/s/shengyu/.conda/envs/hod_env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/global/homes/s/shengyu/.conda/envs/hod_env/lib/python3.11/site-packages/ipykernel/kernelapp.py", 

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [ ]:
data = (data_positions, _ )
randoms = (random_positions_all, _)

In [ ]:
mattrs = get_mesh_attrs(data[0], randoms[0], check=True)
data = ParticleField(*data, attrs=mattrs, exchange=True, backend='jax')
randoms = ParticleField(*randoms, attrs=mattrs, exchange=True, backend='jax')